# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
html = requests.get(url).content

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup
import re
import pandas as pd

In [4]:
soup = BeautifulSoup(html, "lxml")
links = soup.find_all('a')
set(links)
links_list = []
for link in set(links):
    match = re.findall('(href="(.*?)")',str(link)) 
    if match:
        links_list.append(re.sub('(href=)','',(match[0])[1]))
df_links = pd.DataFrame(links_list.copy(),columns = ['links'])
df_links.head(10)

,links
0,http://courses.csail.mit.edu/18.337/2015/docs/...
1,http://www2.isye.gatech.edu/~jeffwu/presentati...
2,#cite_note-36
3,/wiki/Canonical_correlation
4,/wiki/Hierarchical_clustering
5,/wiki/Wikipedia:File_Upload_Wizard
6,https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D1%...
7,#cite_note-19
8,/wiki/Special:RecentChanges
9,/wiki/Artificial_neural_network


## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [5]:
domain = 'http://wikipedia.org'

In [6]:
Absolute_links = [s for l in links_list for s in l.split() if ('https://' in s and '%'not in s) ]
Relative_links = [('{}{}'.format(domain,s)) for l in links_list for s in l.split() if ('https://'not in s and '%'not in s) ]

In [7]:
all_links = Absolute_links + Relative_links
df_both_links = pd.DataFrame({'Relative Links':Relative_links[:57],'Absolute_links':Absolute_links[:57]})
df_both_links.head(10)

,Relative Links,Absolute_links
0,http://wikipedia.orghttp://courses.csail.mit.e...,https://it.wikipedia.org/wiki/Scienza_dei_dati
1,http://wikipedia.orghttp://www2.isye.gatech.ed...,https://www.forbes.com/sites/peterpham/2015/08...
2,http://wikipedia.org#cite_note-36,https://www.wikidata.org/wiki/Special:EntityPa...
3,http://wikipedia.org/wiki/Canonical_correlation,https://www.worldcat.org/oclc/489990740
4,http://wikipedia.org/wiki/Hierarchical_clustering,https://www.springer.com/book/9784431702085
5,http://wikipedia.org/wiki/Wikipedia:File_Uploa...,https://www.mediawiki.org/
6,http://wikipedia.org#cite_note-19,https://foundation.wikimedia.org/wiki/Cookie_s...
7,http://wikipedia.org/wiki/Special:RecentChanges,https://www.forbes.com/sites/gilpress/2013/05/...
8,http://wikipedia.org/wiki/Artificial_neural_ne...,https://donate.wikimedia.org/wiki/Special:Fund...
9,http://wikipedia.org/wiki/Recurrent_neural_net...,https://id.wikipedia.org/wiki/Ilmu_data


## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [8]:
import os.path

In [9]:
try:
    #os.makedirs("wikipedia")        #Crea el dir   
    os.chdir('./wikipedia')         #Cambia el directorio de trabajo actual
    os.getcwd()                     #Muestra el directorio de trabajo actual
except FileExistsError:
    pass

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [10]:
#!pip install python-slugify
from slugify import slugify

In [11]:
def index_page(link):
    try:
        html = requests.get(link).content
        slugy = slugify(link)
        FileName = os.path.join('',"{}.html".format(slugy))
        #file1 = open(FileName, "w") 

        #file1.write(str(html))
        #file1.close()
        with open(FileName, "w+") as f:
            f.write(str(html))
    except:
        pass    

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [12]:
%%time
for e in Absolute_links:
    index_page(e)

Wall time: 45.5 s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [13]:
import multiprocessing

In [ ]:
%%time
pool = multiprocessing.Pool()
result = pool.map(index_page, Absolute_links)
pool.terminate()